## Random Forests

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image
! python -m pip install pydotplus
! pip install graphviz
import statistics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pathways = pd.read_csv('KEGG_7pathways_steps.csv')
pathways = pathways.drop(['Unnamed: 0'], axis=1)
pathways.set_index('studyID',inplace=True)

In [ ]:
pcs = pd.read_csv('PC_final result(use in modeling) (1).csv')
pcs = pcs.iloc[:,:6]
pcs.set_index('studyID',inplace = True)
pathways_2 = pd.read_csv('KEGG_7pathways_steps.csv')
pathways_2 = pathways_2.drop(['Unnamed: 0', 'Group', 'Age', 
       'path:hsa03040-mean',
       'path:hsa03040-variance', 'path:hsa03050-mean',
       'path:hsa03050-variance', 'path:hsa03060-mean',
       'path:hsa03060-variance', 'path:hsa04130-mean',
       'path:hsa04130-variance', 'path:hsa04141-mean',
       'path:hsa04141-variance', 'path:hsa04662-mean',
       'path:hsa04662-variance', 'path:hsa05220-mean',
       'path:hsa05220-variance'], axis=1)
pathways_2.set_index('studyID', inplace = True)
pcs = pcs.merge(pathways_2, how = 'outer', on ='studyID')
pcs.head()

In [ ]:
demographics = pathways.drop(['Steps','path:hsa03040-mean',
       'path:hsa03040-variance', 'path:hsa03050-mean',
       'path:hsa03050-variance', 'path:hsa03060-mean',
       'path:hsa03060-variance', 'path:hsa04130-mean',
       'path:hsa04130-variance', 'path:hsa04141-mean',
       'path:hsa04141-variance', 'path:hsa04662-mean',
       'path:hsa04662-variance', 'path:hsa05220-mean',
       'path:hsa05220-variance'], axis = 1)
demographics.head()

In [ ]:
steps = pathways.drop(['path:hsa03040-mean',
       'path:hsa03040-variance', 'path:hsa03050-mean',
       'path:hsa03050-variance', 'path:hsa03060-mean',
       'path:hsa03060-variance', 'path:hsa04130-mean',
       'path:hsa04130-variance', 'path:hsa04141-mean',
       'path:hsa04141-variance', 'path:hsa04662-mean',
       'path:hsa04662-variance', 'path:hsa05220-mean',
       'path:hsa05220-variance'], axis=1)

Grid for Randomized Search CV - Hyperparameter Tuning

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 400,num=10)]
# Number of features to consider at every split
max_features = ['auto','sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5,7,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2,3,4,5]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

## Demographic Data

In [ ]:
X_dem = demographics[['Group', 'Age', 'Gender', 'BMI']]
Y_dem = demographics['Weight_loss_band']                  
X_train, X_test, y_train, y_test = train_test_split(X_dem,Y_dem, test_size = 0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier()
rf_random= RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
random_forest = RandomForestClassifier(n_estimators = 188,
 min_samples_split = 10,
 min_samples_leaf = 2,
 max_features = 'sqrt',
 max_depth = 50,
 bootstrap = True , random_state = 42)
random_forest.fit(X_train,y_train)
y_pred = random_forest.predict(X_test)
importance = pd.Series(random_forest.feature_importances_, index=X_dem.columns)
importance.nlargest(10).plot(kind='bar');
plt.show()

In [ ]:
random_state = list(range(1,300))
#print(random_state)
d1_train_acc = []
d1_test_acc = []
d1_auc_score = []
d1_prec_score = []
d1_recall_score = []

for i in random_state :
    
    X_dem = demographics[['Group', 'Age', 'Gender', 'BMI']]
    Y_dem = demographics['Weight_loss_band']
    
    X_train, X_test, y_train, y_test= train_test_split(X_dem,Y_dem, test_size = 0.3, random_state = i) 
    
    random_forest= RandomForestClassifier(n_estimators = 188,
     min_samples_split = 10,
     min_samples_leaf = 2,
     max_features = 'sqrt',
     max_depth = 50,
     bootstrap = True , random_state = 42)
    
    random_forest.fit(X_train,y_train)
    y_pred = random_forest.predict(X_test) 

    rf_train_acc = random_forest.score(X_train, y_train)
    rf_test_acc = accuracy_score(y_test,y_pred)
    auc_score = roc_auc_score(y_test,y_pred)
    prec_score = average_precision_score(y_test, y_pred)
    rec_score = recall_score(y_test,y_pred)

    d1_train_acc.append(rf_train_acc)
    d1_test_acc.append(rf_test_acc)
    d1_auc_score.append(auc_score)
    d1_prec_score.append(prec_score)
    d1_recall_score.append(rec_score)
    

print('Mean Training Accuracy:', np.mean(d1_train_acc)) 
print('Mean Test Accuracy:', np.mean(d1_test_acc))
print('Standard Deviation',statistics.stdev(d1_test_acc))

print('AUC score equals',np.mean(d1_auc_score))
print('Standard Deviation for auc score',statistics.stdev(d1_auc_score))
print('Average precision score:', np.mean(d1_prec_score))
print('Average recall score', np.mean(d1_recall_score))

k-fold Cross Validation

In [ ]:
scores = cross_val_score(rf,X_train,y_train, cv=3)
scores               
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

lb = LabelBinarizer()
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])

recall = cross_val_score(rf, X_train, y_train, cv=3, scoring='recall')
print('Recall', np.mean(recall), recall.std())
precision = cross_val_score(rf, X_train, y_train, cv=3, scoring='precision')
print('Precision', np.mean(precision), precision.std())
f1 = cross_val_score(rf, X_train, y_train, cv=3, scoring='f1')
print('F1', np.mean(f1), 'std',f1.std())


## Steps

In [ ]:
X_st = steps[['Group', 'Age', 'Gender', 'BMI', 'Steps']]
Y_st =steps['Weight_loss_band']                  
X_train, X_test, y_train, y_test = train_test_split(X_st,Y_st, test_size = 0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier()
rf_random= RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
random_forest = RandomForestClassifier(n_estimators = 104,
 min_samples_split = 10,
 min_samples_leaf = 3,
 max_features = 'auto',
 max_depth = 2,
 bootstrap =  False)
random_forest.fit(X_train,y_train)
y_pred = random_forest.predict(X_test)
importance= pd.Series(random_forest.feature_importances_, index=X_st.columns)
importance.nlargest(5).plot(kind='bar');
plt.show()

In [ ]:
random_state = list(range(1,300))

d1_train_acc = []
d1_test_acc = []
d1_auc_score = []
d1_prec_score = []
d1_recall_score = []

for i in random_state :
    
    X_steps = pathways[['Group', 'Age', 'Gender', 'BMI',  'Steps']]
    Y_steps = steps['Weight_loss_band']
    
    X_train, X_test, y_train, y_test= train_test_split(X_steps,Y_steps, test_size = 0.2, random_state = i) 
    
    random_forest= RandomForestClassifier(n_estimators = 104,
     min_samples_split = 10,
     min_samples_leaf = 3,
     max_features = 'auto',
     max_depth = 2,
     bootstrap =  False)
    
    random_forest.fit(X_train,y_train)
    y_pred = random_forest.predict(X_test) 

    rf_train_acc = random_forest.score(X_train, y_train)
    rf_test_acc = accuracy_score(y_test,y_pred)
    auc_score = roc_auc_score(y_test,y_pred)
    prec_score = average_precision_score(y_test, y_pred)
    rec_score = recall_score(y_test,y_pred)

    d1_train_acc.append(rf_train_acc)
    d1_test_acc.append(rf_test_acc)
    d1_auc_score.append(auc_score)
    d1_prec_score.append(prec_score)
    d1_recall_score.append(rec_score)
    

print('Mean Training Accuracy:', np.mean(d1_train_acc)) 
print('Mean Test Accuracy:', np.mean(d1_test_acc))
print(statistics.stdev(d1_test_acc))

print('AUC score equals',np.mean(d1_auc_score))
print('Standard Deviation for auc score',statistics.stdev(d1_auc_score))
print('Average precision score:', np.mean(d1_prec_score))
print('Average recall score', np.mean(d1_recall_score))

In [ ]:
scores = cross_val_score(rf,X_train,y_train, cv=3)
scores               
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

lb = LabelBinarizer()
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])

recall = cross_val_score(rf, X_train, y_train, cv=3, scoring='recall')
print('Recall', np.mean(recall), recall.std())
precision = cross_val_score(rf, X_train, y_train, cv=3, scoring='precision')
print('Precision', np.mean(precision), precision.std())
f1 = cross_val_score(rf, X_train, y_train, cv=3, scoring='f1')
print('F1', np.mean(f1), 'std',f1.std())

## Pathways

In [ ]:
X_pa = pathways[['Group', 'Age', 'Gender', 'BMI', 'Steps',
       'path:hsa03040-mean', 'path:hsa03040-variance', 'path:hsa03050-mean',
       'path:hsa03050-variance', 'path:hsa03060-mean',
       'path:hsa03060-variance', 'path:hsa04130-mean',
       'path:hsa04130-variance', 'path:hsa04141-mean',
       'path:hsa04141-variance', 'path:hsa04662-mean',
       'path:hsa04662-variance', 'path:hsa05220-mean',
       'path:hsa05220-variance']]
Y_pa =pathways['Weight_loss_band']                  
X_train, X_test, y_train, y_test = train_test_split(X_pa,Y_pa, test_size = 0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier()
rf_random= RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
random_forest = RandomForestClassifier(n_estimators = 273,
 min_samples_split = 5,
 min_samples_leaf = 3,
 max_features = 'sqrt',
 criterion = 'entropy',
 max_depth = 2,
 bootstrap = True)
random_forest.fit(X_train,y_train)
y_pred = random_forest.predict(X_test)
importance = pd.Series(random_forest.feature_importances_, index=X_pa.columns)
importance.nlargest(10).plot(kind='bar');
plt.show()

In [ ]:
random_state = list(range(1,300))

d1_train_acc = []
d1_test_acc = []
d1_auc_score = []
d1_prec_score = []
d1_recall_score = []


for i in random_state :
    
    X_pa = pathways[['Group', 'Age', 'Gender', 'BMI', 'Steps',
       'path:hsa03040-mean', 'path:hsa03040-variance', 'path:hsa03050-mean',
       'path:hsa03050-variance', 'path:hsa03060-mean',
       'path:hsa03060-variance', 'path:hsa04130-mean',
       'path:hsa04130-variance', 'path:hsa04141-mean',
       'path:hsa04141-variance', 'path:hsa04662-mean',
       'path:hsa04662-variance', 'path:hsa05220-mean',
       'path:hsa05220-variance']]
    Y_pa = pathways['Weight_loss_band']
    
    X_train, X_test, y_train, y_test= train_test_split(X_pa,Y_pa, test_size = 0.2, random_state = i) 
    
    random_forest= RandomForestClassifier(n_estimators = 104,
     min_samples_split = 10,
     min_samples_leaf = 3,
     max_features = 'auto',
     max_depth = 2,
     bootstrap =  False)
    
    random_forest.fit(X_train,y_train)
    y_pred = random_forest.predict(X_test) 

    rf_train_acc = random_forest.score(X_train, y_train)
    rf_test_acc = accuracy_score(y_test,y_pred)
    auc_score = roc_auc_score(y_test,y_pred)
    prec_score = average_precision_score(y_test, y_pred)
    rec_score = recall_score(y_test,y_pred)

    d1_train_acc.append(rf_train_acc)
    d1_test_acc.append(rf_test_acc)
    d1_auc_score.append(auc_score)
    d1_prec_score.append(prec_score)
    d1_recall_score.append(rec_score)
    

print('Mean Training Accuracy:', np.mean(d1_train_acc)) 
print('Mean Test Accuracy:', np.mean(d1_test_acc))
print(statistics.stdev(d1_test_acc))

print('AUC score equals',np.mean(d1_auc_score))
print('Standard Deviation for auc score',statistics.stdev(d1_auc_score))
print('Average precision score:', np.mean(d1_prec_score))
print('Average recall score', np.mean(d1_recall_score))

In [ ]:
scores = cross_val_score(rf,X_train,y_train, cv=3)
scores               
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
lb = LabelBinarizer()
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])

recall = cross_val_score(rf, X_train, y_train, cv=3, scoring='recall')
print('Recall', np.mean(recall), recall.std())
precision = cross_val_score(rf, X_train, y_train, cv=3, scoring='precision')
print('Precision', np.mean(precision), precision.std())
f1 = cross_val_score(rf, X_train, y_train, cv=3, scoring='f1')
print('F1', np.mean(f1), 'std',f1.std())

## Extra model without some pathways

In [ ]:
random_state = list(range(1,300))

d1_train_acc = []
d1_test_acc = []
d1_auc_score = []
d1_prec_score = []
d1_recall_score = []

for i in random_state :
    
    X_pa = pathways[['Group', 'Age', 'Gender', 'BMI', 'Steps',
       'path:hsa03040-mean', 'path:hsa03040-variance', 'path:hsa03050-mean',
       'path:hsa03050-variance', 'path:hsa03060-mean',
       'path:hsa03060-variance']]
    Y_pa = pathways['Weight_loss_band']
    
    X_train, X_test, y_train, y_test= train_test_split(X_pa,Y_pa, test_size = 0.2, random_state = i) 
    
    random_forest= RandomForestClassifier(n_estimators = 104,
     min_samples_split = 10,
     min_samples_leaf = 3,
     max_features = 'auto',
     max_depth = 2,
     bootstrap =  False)
    
    random_forest.fit(X_train,y_train)
    y_pred = random_forest.predict(X_test) 

    rf_train_acc = random_forest.score(X_train, y_train)
    rf_test_acc = accuracy_score(y_test,y_pred)
    auc_score = roc_auc_score(y_test,y_pred)
    prec_score = average_precision_score(y_test, y_pred)
    rec_score = recall_score(y_test,y_pred)

    d1_train_acc.append(rf_train_acc)
    d1_test_acc.append(rf_test_acc)
    d1_auc_score.append(auc_score)
    d1_prec_score.append(prec_score)
    d1_recall_score.append(rec_score)

print('Mean Training Accuracy:', np.mean(d1_train_acc)) 
print('Mean Test Accuracy:', np.mean(d1_test_acc))
print(statistics.stdev(d1_test_acc))

print('AUC score equals',np.mean(d1_auc_score))
print('Standard Deviation for auc score',statistics.stdev(d1_auc_score))
print('Average precision score:', np.mean(d1_prec_score))
print('Average recall score', np.mean(d1_recall_score))

In [ ]:
scores = cross_val_score(rf,X_train,y_train, cv=5)
scores               
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
lb = LabelBinarizer()
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])

recall = cross_val_score(rf, X_train, y_train, cv=3, scoring='recall')
print('Recall', np.mean(recall), recall.std())
precision = cross_val_score(rf, X_train, y_train, cv=3, scoring='precision')
print('Precision', np.mean(precision), precision.std())
f1 = cross_val_score(rf, X_train, y_train, cv=3, scoring='f1')
print('F1', np.mean(f1), 'std',f1.std())

## PCA

In [ ]:
X= pcs[['PC1', 'PC2', 'PC3', 'PC4', 'PC5','Steps','Gender','BMI']]
Y = pcs['Weight_loss_band']                
X_train, X_test, y_train, y_test= train_test_split(X,Y, test_size = 0.3, random_state=42)

In [ ]:
rf = RandomForestClassifier()
rf_random= RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
random_forest= RandomForestClassifier(n_estimators = 315,
 min_samples_split = 7,
 min_samples_leaf = 2,
 max_features = 'auto',
 max_depth = 14,
 bootstrap = False , random_state = 42)
random_forest.fit(X_train,y_train)
y_pred = random_forest.predict(X_test)
importance = pd.Series(random_forest.feature_importances_, index=X.columns)
importance.nlargest(10).plot(kind='bar');
plt.show()

In [ ]:
random_state = list(range(1,300))

d1_train_acc = []
d1_test_acc = []
d1_auc_score = []
d1_prec_score = []
d1_recall_score = []

for i in random_state :
    
    X_pcs = pcs[['PC3', 'PC2', 'PC5', 'Gender', 'BMI', 'Steps']]
    Y_pcs = pcs['Weight_loss_band']
    
    X_train, X_test, y_train, y_test= train_test_split(X_pcs,Y_pcs, test_size = 0.2, random_state = i) 
    
    random_forest= RandomForestClassifier(n_estimators = 315,
     min_samples_split = 7,
     min_samples_leaf = 2,
     max_features = 'auto',
     max_depth = 14,
     bootstrap = False)
    
    random_forest.fit(X_train,y_train)
    y_pred = random_forest.predict(X_test) 

    rf_train_acc = random_forest.score(X_train, y_train)
    rf_test_acc = accuracy_score(y_test,y_pred)
    auc_score = roc_auc_score(y_test,y_pred)
    prec_score = average_precision_score(y_test, y_pred)
    rec_score = recall_score(y_test,y_pred)

    d1_train_acc.append(rf_train_acc)
    d1_test_acc.append(rf_test_acc)
    d1_auc_score.append(auc_score)
    d1_prec_score.append(prec_score)
    d1_recall_score.append(rec_score)

print('Mean Training Accuracy:', np.mean(d1_train_acc)) 
print('Mean Test Accuracy:', np.mean(d1_test_acc))
print(statistics.stdev(d1_test_acc))

print('AUC score equals',np.mean(d1_auc_score))
print('Standard Deviation for auc score',statistics.stdev(d1_auc_score))
print('Average precision score:', np.mean(d1_prec_score))
print('Average recall score', np.mean(d1_recall_score))

In [ ]:
scores = cross_val_score(rf,X_train,y_train, cv=5)
scores               
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
lb = LabelBinarizer()
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])

recall = cross_val_score(rf, X_train, y_train, cv=3, scoring='recall')
print('Recall', np.mean(recall), recall.std())
precision = cross_val_score(rf, X_train, y_train, cv=3, scoring='precision')
print('Precision', np.mean(precision), precision.std())
f1 = cross_val_score(rf, X_train, y_train, cv=3, scoring='f1')
print('F1', np.mean(f1), 'std',f1.std())